# Hummingbot trades graph

This notebook parse hummingbot logs and then plot sell and buy operations

In [1]:
import plotly.graph_objects as go
import pandas as pd
from core.log_parser import log_parser
import configparser
from core.binance_client import Binance
import plotly.io as pio
pio.renderers.default = 'browser'

Read API token and secret  from settings file

In [2]:
config = configparser.ConfigParser()
config.read("settings.ini")

api_key = config["Binance"]["api_key"]
secret = config["Binance"]["secret"]

binance = Binance(api_key, secret)

Read logs from humming bot files directory

In [3]:
symbol = 'ETHUSDT'

In [4]:
bot_logs = log_parser("/home/helllynx/SSH/GERM/root/hummingbot-docker/hummingbot_files/hummingbot_logs/logs_conf_pure_mm_2_ping_pong.log")

ticks = binance.get_ticks(symbol, from_time=bot_logs.iloc[0].time, interval='5m')
ticks = binance.klines_to_dataframe(ticks)
ticks['Open time'] = pd.to_datetime(ticks['Open time'], unit='ms')
ticks['Close time'] = pd.to_datetime(ticks['Close time'], unit='ms')
ticks['Spread'] = ticks.apply(lambda row: row['High'] - row['Low'], axis=1)
ticks['Spread Percentage'] = ticks.apply(lambda row: row['Spread']/row['Close'], axis=1)

bot_logs['time'] = pd.to_datetime(bot_logs['time'])

Plot graph

In [8]:
buys = bot_logs[bot_logs['operation']=='buy']
sells = bot_logs[bot_logs['operation']=='sell']

fig = go.Figure(data=[go.Candlestick(
                x=ticks['Open time'],
                open=ticks['Open'],
                high=ticks['High'],
                low=ticks['Low'],
                close=ticks['Close'])])
fig.update_layout(xaxis_rangeslider_visible=False)


fig.add_trace(go.Scatter(x=buys['time'], y=buys['value_quote'], mode='markers', marker_symbol='triangle-up', marker_color="lightskyblue", marker_size=12, name = 'buy'))
fig.add_trace(go.Scatter(x=sells['time'], y=sells['value_quote'], mode='markers',  marker_symbol='triangle-down', marker_color="midnightblue", marker_size=12, name = 'sell'))
fig.show()

Calculate spread for each tick

In [6]:
ticks.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore.,Spread,Spread Percentage
0,2021-02-28 08:35:00,1362.19,1369.00,1361.54,1362.99,3587.53209,2021-02-28 08:39:59.999,4894158.02337090,3915,1953.44481000,2665108.50855000,0,7.46,0.005473
1,2021-02-28 08:40:00,1362.99,1365.06,1358.08,1363.25,3850.05332,2021-02-28 08:44:59.999,5241711.70750240,3262,1892.12793000,2576071.92110070,0,6.98,0.005120
2,2021-02-28 08:45:00,1363.08,1363.22,1355.41,1356.47,2721.39304,2021-02-28 08:49:59.999,3698501.50884180,2895,1330.05685000,1807673.11693650,0,7.81,0.005758
3,2021-02-28 08:50:00,1356.47,1358.93,1354.66,1358.13,2340.63395,2021-02-28 08:54:59.999,3175647.12505900,2719,1121.18305000,1521029.39745930,0,4.27,0.003144
4,2021-02-28 08:55:00,1358.07,1362.14,1356.25,1359.59,2871.84706,2021-02-28 08:59:59.999,3902097.50136000,2754,1447.15784000,1965975.20161380,0,5.89,0.004332


In [7]:
bot_logs.head()

,time,operation,base,value_base,quote,value_quote
0,2021-02-28 08:32:39+00:00,buy,ETH,0.1,USDT,1365.623
1,2021-02-28 09:02:07+00:00,sell,ETH,0.1,USDT,1364.604
